In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from scipy import stats
import numpy as np
from sklearn.ensemble import IsolationForest
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

In [2]:
df = pd.read_csv(r'../dataset/predictive_maintenance.csv')
df.head(2)

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure


In [3]:
print(df.head())

print(df.info())

   UDI Product ID Type  Air temperature [K]  Process temperature [K]  \
0    1     M14860    M                298.1                    308.6   
1    2     L47181    L                298.2                    308.7   
2    3     L47182    L                298.1                    308.5   
3    4     L47183    L                298.2                    308.6   
4    5     L47184    L                298.2                    308.7   

   Rotational speed [rpm]  Torque [Nm]  Tool wear [min]  Target Failure Type  
0                    1551         42.8                0       0   No Failure  
1                    1408         46.3                3       0   No Failure  
2                    1498         49.4                5       0   No Failure  
3                    1433         39.5                7       0   No Failure  
4                    1408         40.0                9       0   No Failure  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (tota

In [4]:
cols = ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']

X = StandardScaler().fit_transform(df[cols])

dbscan = DBSCAN(eps=0.5, min_samples=10)
df['DBSCAN_Cluster'] = dbscan.fit_predict(X)

outliers = df[df['DBSCAN_Cluster'] == -1]

outliers.to_csv(r'../dataset/outliers_dbscan.csv', index=False)

print(outliers.head())

    UDI Product ID Type  Air temperature [K]  Process temperature [K]  \
14   15     L47194    L                298.6                    309.2   
50   51     L47230    L                298.9                    309.1   
62   63     L47242    L                298.8                    309.0   
69   70     L47249    L                298.9                    309.0   
70   71     M14930    M                298.9                    309.0   

    Rotational speed [rpm]  Torque [Nm]  Tool wear [min]  Target  \
14                    2035         19.6               40       0   
50                    2861          4.6              143       1   
62                    1829         22.9              172       0   
69                    1410         65.7              191       1   
70                    1924         22.6              193       0   

     Failure Type  DBSCAN_Cluster  
14     No Failure              -1  
50  Power Failure              -1  
62     No Failure              -1  
69  Powe

In [5]:
cols = ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']

z = np.abs(stats.zscore(df[cols]))
z_threshold = 3
outliers_z = np.where(z > z_threshold)

Q1 = df[cols].quantile(0.25)
Q3 = df[cols].quantile(0.75)
IQR = Q3 - Q1
outliers_iqr = ((df[cols] < (Q1 - 1.5 * IQR)) | (df[cols] > (Q3 + 1.5 * IQR))).any(axis=1)

print(f"Number of outliers detected by Z-Score method: {len(np.unique(outliers_z[0]))}")
print(f"Number of outliers detected by IQR method: {outliers_iqr.sum()}")

df.iloc[outliers_z[0]].to_csv(r'../dataset/z_score_outliers.csv', index=False)
df[outliers_iqr].to_csv(r'../dataset/iqr_outliers.csv', index=False)

Number of outliers detected by Z-Score method: 178
Number of outliers detected by IQR method: 459


In [6]:
cols = ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']
X = df[cols]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

iso_forest = IsolationForest(n_estimators=100, contamination='auto', random_state=42)

iso_forest.fit(X_scaled)


df['anomaly'] = iso_forest.predict(X_scaled)

anomalies = df[df['anomaly'] == -1]

print(anomalies)

       UDI Product ID Type  Air temperature [K]  Process temperature [K]  \
14      15     L47194    L                298.6                    309.2   
50      51     L47230    L                298.9                    309.1   
69      70     L47249    L                298.9                    309.0   
70      71     M14930    M                298.9                    309.0   
101    102     L47281    L                298.8                    308.8   
...    ...        ...  ...                  ...                      ...   
9951  9952     L57131    L                298.2                    307.8   
9970  9971     H39384    H                298.4                    308.1   
9974  9975     L57154    L                298.6                    308.2   
9988  9989     L57168    L                298.9                    308.6   
9991  9992     M24851    M                298.9                    308.4   

      Rotational speed [rpm]  Torque [Nm]  Tool wear [min]  Target  \
14               

In [7]:
X = df[['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

input_dim = X_scaled.shape[1]
encoding_dim = 14  

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="relu")(input_layer)
decoder = Dense(input_dim, activation="linear")(encoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

autoencoder.compile(optimizer=Adam(lr=0.001), loss='mean_squared_error')

autoencoder.fit(X_scaled, X_scaled, epochs=100, batch_size=32, shuffle=True, validation_split=0.2, verbose=1)


reconstructed = autoencoder.predict(X_scaled)
mse = np.mean(np.power(X_scaled - reconstructed, 2), axis=1)

threshold = np.quantile(mse, 0.95)  

outliers = mse > threshold
print("Number of anomalies detected:", np.sum(outliers))

Epoch 1/100



250/250 [==============================] - 1s 2ms/step - loss: 0.5455 - val_loss: 0.1783
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.1353 - val_loss: 0.0674
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.0519 - val_loss: 0.0360
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.0272 - val_loss: 0.0235
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.0173 - val_loss: 0.0145
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.0120 - val_loss: 0.0109
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.0088 - val_loss: 0.0086
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.0067 - val_loss: 0.0068
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.0052 - val_loss: 0.0053
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.0039 - val_loss: 0.0041
E

In [8]:
X = df[['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']].values

knn = NearestNeighbors(n_neighbors=5)  

knn.fit(X)

distances, indices = knn.kneighbors(X)

avg_distance = np.mean(distances, axis=1)

threshold = np.percentile(avg_distance, 95)  

anomalies = avg_distance > threshold

print(f"Number of anomalies detected: {np.sum(anomalies)}")

Number of anomalies detected: 500


In [9]:
X = df[['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=2)  
X_pca = pca.fit_transform(X_scaled)

X_inverse = pca.inverse_transform(X_pca)


reconstruction_error = np.sum(np.square(X_scaled - X_inverse), axis=1)

threshold = np.percentile(reconstruction_error, 95) 

anomalies = reconstruction_error > threshold

print(f"Number of anomalies detected: {np.sum(anomalies)}")

Number of anomalies detected: 500
